# Mempersiapkan environment

##a. Install pyspark package
Untuk menjalankan notebook ini di Google colab, langkah pertama yang perlu dilakukan adalah menginstall package `pyspark`, karena package tersebut tidak disertakan secara default.

Langkah ini perlu dilakukan setiap membuka session/notebook baru.

Instalasi kita lakukan dengan perintah `pip`

In [ ]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=8c3f21c3fb64496332e55192de0fc68c36127f6c470f10a19ed03fc62ef6b4ab
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark



##b. Membuat spark session

Import library yang akan digunakan.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Inisialisasi spark session untuk berinteraksi dengan Spark cluster

In [ ]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

#1.Lab 02 - Membuat DataFrame

DataFrame dapat dibuat dengan banyak cara, di antaranya :
- Dari python object, misalnya array/list, dictionary, pandas dataframe, dll
- Dari file : csv, json, dll
- Dari HDFS
- Dari RDD
- dll.

##1.1. Create From Array

In [ ]:
mydata = (('DKI JAKARTA',15328),
('JAWA BARAT',1320),
('JAWA TENGAH',1030),
('DI YOGYAKARTA',1174),
('JAWA TIMUR',813),
('BANTEN',1237))

df_from_array = spark.createDataFrame(mydata).toDF("province", "density")

df_from_array.show()

+-------------+-------+
|     province|density|
+-------------+-------+
|  DKI JAKARTA|  15328|
|   JAWA BARAT|   1320|
|  JAWA TENGAH|   1030|
|DI YOGYAKARTA|   1174|
|   JAWA TIMUR|    813|
|       BANTEN|   1237|
+-------------+-------+



##1.2. Create from Pandas DataFrame

Dowload file

In [ ]:
!wget https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/penduduk2015.csv

--2023-06-16 02:56:59--  https://www.dropbox.com/s/65mohvoyjshtqa9/penduduk2015.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/65mohvoyjshtqa9/penduduk2015.csv [following]
--2023-06-16 02:56:59--  https://www.dropbox.com/s/raw/65mohvoyjshtqa9/penduduk2015.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce38357161e5fb0072e163095ba.dl.dropboxusercontent.com/cd/0/inline/B-GdtwCO3zPfA7gdzREuo7NuDSsaFP40NBEAhCc9MEVY8QYHwmBSJ_My2e3v6I8mPGoOt5HZYrkaREHgslE6jCz8PzBY4Ggy6hwgCbYMZM1vMBXoPtnf4dag-BO7KOzlsx3EQ1LUOcnSnpPKdziKEJc6bWU6oNMFA1EHUoqXFx-v9Q/file# [following]
--2023-06-16 02:57:00--  https://uce38357161e5fb0072e163095ba.dl.dropboxusercontent.com/cd/0/inline/B-GdtwCO3zPfA7gdzREuo7NuDSsaFP40NBEAhCc9MEVY8QYHwmBSJ_My2e3v6I8mPGo

Create pandas dataframe dari file csv tersebut

In [ ]:
import pandas as pd

pddf = pd.read_csv('penduduk2015.csv')
pddf

,propinsi,penduduk
0,DKI JAKARTA,15328
1,JAWA BARAT,1320
2,JAWA TENGAH,1030
3,DI YOGYAKARTA,1174
4,JAWA TIMUR,813
5,BANTEN,1237


Ubah ke Spark dataframe

In [ ]:
df_from_pandas = spark.createDataFrame(pddf)
df_from_pandas.show()

+-------------+--------+
|     propinsi|penduduk|
+-------------+--------+
|  DKI JAKARTA|   15328|
|   JAWA BARAT|    1320|
|  JAWA TENGAH|    1030|
|DI YOGYAKARTA|    1174|
|   JAWA TIMUR|     813|
|       BANTEN|    1237|
+-------------+--------+



##1.3. Create from csv

Kita juga bisa me-load langsung file csv tersebut ke Spark dataframe

In [ ]:
df_from_csv = spark.read.csv("penduduk2015.csv", header=True)
df_from_csv.show()

+-------------+--------+
|     propinsi|penduduk|
+-------------+--------+
|  DKI JAKARTA|   15328|
|   JAWA BARAT|    1320|
|  JAWA TENGAH|    1030|
|DI YOGYAKARTA|    1174|
|   JAWA TIMUR|     813|
|       BANTEN|    1237|
+-------------+--------+



##1.4. Create from JSON

In [ ]:
!wget https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/penduduk2015.csv

--2023-06-16 02:58:01--  https://www.dropbox.com/s/8wpyzukkmmn3t6g/penduduk2015.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/8wpyzukkmmn3t6g/penduduk2015.json [following]
--2023-06-16 02:58:02--  https://www.dropbox.com/s/raw/8wpyzukkmmn3t6g/penduduk2015.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc11be2ee1fa76b97d34791723d0.dl.dropboxusercontent.com/cd/0/inline/B-Eno0sghvbSBxmm-HFCNYPpkZYY9GMLlNk4KSo5rDqs09bIELxf1ZDQPkARtkQGKWbd7Jf6f83Fqkzns0D016b96LIvp7lgnv6grg8X5uDE5zPl7LfPALhbKnehAxNs_oHjXRHM8dA4jwLEXVEx-NHvx7P5y_rtanMuiQAdAI2rLA/file# [following]
--2023-06-16 02:58:02--  https://uc11be2ee1fa76b97d34791723d0.dl.dropboxusercontent.com/cd/0/inline/B-Eno0sghvbSBxmm-HFCNYPpkZYY9GMLlNk4KSo5rDqs09bIELxf1ZDQPkARtkQGK

Tampilkan isi file dengan perintah `cat`

In [ ]:
!cat penduduk2015.json

[
 {
   "propinsi": "DKI JAKARTA",
   "penduduk": 15328
 },
 {
   "propinsi": "JAWA BARAT",
   "penduduk": 1320
 },
 {
   "propinsi": "JAWA TENGAH",
   "penduduk": 1030
 },
 {
   "propinsi": "DI YOGYAKARTA",
   "penduduk": 1174
 },
 {
   "propinsi": "JAWA TIMUR",
   "penduduk": 813
 },
 {
   "propinsi": "BANTEN",
   "penduduk": 1237
 }
]

Untuk membaca multiline JSON, set parameter `multiline` = True

In [ ]:
dfj = spark.read.json("penduduk2015.json", multiLine=True)
dfj.show()

+--------+-------------+
|penduduk|     propinsi|
+--------+-------------+
|   15328|  DKI JAKARTA|
|    1320|   JAWA BARAT|
|    1030|  JAWA TENGAH|
|    1174|DI YOGYAKARTA|
|     813|   JAWA TIMUR|
|    1237|       BANTEN|
+--------+-------------+



#2.Lab 03 - Explorasi DataFrame

Dalam latihan ini kita akan mencoba berbagai operasi pada Spark DataFrame untuk melakukan eksplorasi data.

Kita akan menggunakan data kepadatan penduduk per propinsi.

In [ ]:
#!wget https://www.dropbox.com/s/4ozzaa2yvy2kkza/indonesia2013-2015.csv
!wget https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/indonesia2013-2015.csv

--2023-06-16 02:58:48--  https://www.dropbox.com/s/4ozzaa2yvy2kkza/indonesia2013-2015.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4ozzaa2yvy2kkza/indonesia2013-2015.csv [following]
--2023-06-16 02:58:48--  https://www.dropbox.com/s/raw/4ozzaa2yvy2kkza/indonesia2013-2015.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce38c0742af78bc9a60e84e2859.dl.dropboxusercontent.com/cd/0/inline/B-GDHcMiawTz6D87lHEFvw4XvWRwUFKsVOCWDc4Hv4uZEBv333uNzjS7bNV_pGC_vWqeC0PBCtA6Lgq_yyr-6zPkbSS1UhdHKSg1o8b3xLY8-2Soj_smU3jpk0_fInhPK9-HiWT63sXsmrdODkxemOTFpsgIHO8MTsKaR6j-2Jl5BA/file# [following]
--2023-06-16 02:58:48--  https://uce38c0742af78bc9a60e84e2859.dl.dropboxusercontent.com/cd/0/inline/B-GDHcMiawTz6D87lHEFvw4XvWRwUFKsVOCWDc4Hv4uZEBv33

Kita gunakan magic command untuk melihat ukuran dan isi file (karena file kita cukup kecil).

In [ ]:
%ls -al indonesia2013-2015.csv

-rw-r--r-- 1 root root 2771 Jun 16 02:58 indonesia2013-2015.csv


In [ ]:
%cat indonesia2013-2015.csv

province,year,density,timezone
ACEH,2015,86,WIB
SUMATERA UTARA,2015,191,WIB
SUMATERA BARAT,2015,124,WIB
RIAU,2015,73,WIB
JAMBI,2015,68,WIB
KEP. RIAU,2015,241,WIB
KEP. BANGKA BELITUNG,2015,84,WIB
BENGKULU,2015,94,WIB
SUMATERA SELATAN,2015,88,WIB
LAMPUNG,2015,234,WIB
BANTEN,2015,1237,WIB
JAWA BARAT,2015,1320,WIB
DKI JAKARTA,2015,15328,WIB
JAWA TENGAH,2015,1030,WIB
DI YOGYAKARTA,2015,1174,WIB
JAWA TIMUR,2015,813,WIB
KALIMANTAN BARAT,2015,33,WIB
KALIMANTAN TENGAH,2015,16,WIB
KALIMANTAN UTARA,2015,9,WITA
KALIMANTAN SELATAN,2015,103,WITA
KALIMANTAN TIMUR,2015,27,WITA
BALI,2015,718,WITA
NUSA TENGGARA BARAT,2015,260,WITA
NUSA TENGGARA TIMUR,2015,105,WITA
GORONTALO,2015,101,WITA
SULAWESI UTARA,2015,174,WITA
SULAWESI TENGAH,2015,47,WITA
SULAWESI BARAT,2015,76,WITA
SULAWESI SELATAN,2015,182,WITA
SULAWESI TENGGARA,2015,66,WITA
MALUKU UTARA,2015,36,WIT
MALUKU,2015,36,WIT
PAPUA,2015,10,WIT
PAPUA BARAT,2015,9,WIT
ACEH,2014,85,WIB
SUMATERA UTARA,2014,189,WIB
SUMATERA BARAT,2014,122,WIB
RIAU,2014,71,WI

Karena data yang kita load sudah bersih, kita akan set inferSchema = True agar Spark menyesuaikan tipe kolom dengan datanya.

In [ ]:
df = spark.read.csv("indonesia2013-2015.csv",header=True,inferSchema=True)

###2.1 Melihat sekilas

####Menampilkan beberapa baris

Biasanya kita menampilkan beberapa baris data untuk mengecek format dan konten dataframe yang kita buat.

Untuk menampilkan beberapa baris dari dataframe, kita bisa gunakan perintah ``show(n)`` untuk menampilkan n baris pertama, atau ``first()`` untuk menampilkan 1 baris pertama saja.

In [ ]:
df.show(5)
df.first()

+--------------+----+-------+--------+
|      province|year|density|timezone|
+--------------+----+-------+--------+
|          ACEH|2015|     86|     WIB|
|SUMATERA UTARA|2015|    191|     WIB|
|SUMATERA BARAT|2015|    124|     WIB|
|          RIAU|2015|     73|     WIB|
|         JAMBI|2015|     68|     WIB|
+--------------+----+-------+--------+
only showing top 5 rows



Row(province='ACEH', year=2015, density=86, timezone='WIB')

####Menampilkan jumlah kolom

In [ ]:
df.columns

['province', 'year', 'density', 'timezone']

####Menampilkan total records

In [ ]:
df.count()

102

####Menampilkan skema

Untuk menampilkan skema dataframe, gunakan fungsi `printSchema()`



In [ ]:
df.printSchema()

root
 |-- province: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- density: integer (nullable = true)
 |-- timezone: string (nullable = true)



Akses atribut `columns` untuk menampilkan list nama kolom


In [ ]:
df.columns

['province', 'year', 'density', 'timezone']

Akses atribut `dtypes` untuk menampilkan list nama kolom beserta data type masing-masing kolom tersebut

In [ ]:
df.dtypes

[('province', 'string'),
 ('year', 'int'),
 ('density', 'int'),
 ('timezone', 'string')]

####Menampilkan summary statistik

Fungsi `describe()` digunakan untuk menampilkan summary statistik dari seluruh kolom.

Jangan lupa memanggil fungsi `show()` untuk menampilkan hasilnya.

In [ ]:
df.describe().show()

+-------+--------------+-----------------+------------------+--------+
|summary|      province|             year|           density|timezone|
+-------+--------------+-----------------+------------------+--------+
|  count|           102|              102|               101|     102|
|   mean|          null|           2014.0| 710.2475247524752|    null|
| stddev|          null|0.820528687330573|2570.8287045462234|    null|
|    min|          ACEH|             2013|                 8|     WIB|
|    max|SUMATERA UTARA|             2015|             15328|    WITA|
+-------+--------------+-----------------+------------------+--------+



Untuk menampilkan statistik dari salah satu kolom saja, gunakan nama kolom yang akan ditampilkan sebagai parameter. Misalnya `describe('column1')`

In [ ]:
df.describe("density").show()

+-------+------------------+
|summary|           density|
+-------+------------------+
|  count|               101|
|   mean| 710.2475247524752|
| stddev|2570.8287045462234|
|    min|                 8|
|    max|             15328|
+-------+------------------+



###2.2 Filtering



Kita dapat melakukan filtering terhadap spark dataframe, berdasar kolom atau baris

####Memilih kolom tertentu

Untuk menampilkan kolom tertentu, digunakan fungsi `select('nama_kolom')`


In [ ]:
df.select("year").show(10)

+----+
|year|
+----+
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
+----+
only showing top 10 rows



Untuk memilih beberapa kolom, gunakan tanda koma sebagai pemisah

In [ ]:
df.select("province","density").show(10)

+--------------------+-------+
|            province|density|
+--------------------+-------+
|                ACEH|     86|
|      SUMATERA UTARA|    191|
|      SUMATERA BARAT|    124|
|                RIAU|     73|
|               JAMBI|     68|
|           KEP. RIAU|    241|
|KEP. BANGKA BELITUNG|     84|
|            BENGKULU|     94|
|    SUMATERA SELATAN|     88|
|             LAMPUNG|    234|
+--------------------+-------+
only showing top 10 rows



####Memilih records / baris

Untuk memilih baris dengan kondisi tertentu, gunakan fungsi `filter(<kondisi>)`



In [ ]:
df.filter(df.density > 5000).show()

+-----------+----+-------+--------+
|   province|year|density|timezone|
+-----------+----+-------+--------+
|DKI JAKARTA|2015|  15328|     WIB|
|DKI JAKARTA|2014|  15173|     WIB|
|DKI JAKARTA|2013|  15015|     WIB|
+-----------+----+-------+--------+



Untuk menggunakan kondisi berupa operasi string, dapat digunakan fungsi-fungsi dari `pyspark.sql.Column` yang terkait string, misalnya `contains()`, `startswith()`, `endswith()`

In [ ]:
df.filter(df.province.contains('TENGGARA')).show(5)
df.filter(df.province.startswith('SU')).show(5)
df.filter(df.province.endswith('BARAT')).show(5)

+-------------------+----+-------+--------+
|           province|year|density|timezone|
+-------------------+----+-------+--------+
|NUSA TENGGARA BARAT|2015|    260|    WITA|
|NUSA TENGGARA TIMUR|2015|    105|    WITA|
|  SULAWESI TENGGARA|2015|     66|    WITA|
|NUSA TENGGARA BARAT|2014|    257|    WITA|
|NUSA TENGGARA TIMUR|2014|    103|    WITA|
+-------------------+----+-------+--------+
only showing top 5 rows

+----------------+----+-------+--------+
|        province|year|density|timezone|
+----------------+----+-------+--------+
|  SUMATERA UTARA|2015|    191|     WIB|
|  SUMATERA BARAT|2015|    124|     WIB|
|SUMATERA SELATAN|2015|     88|     WIB|
|  SULAWESI UTARA|2015|    174|    WITA|
| SULAWESI TENGAH|2015|     47|    WITA|
+----------------+----+-------+--------+
only showing top 5 rows

+-------------------+----+-------+--------+
|           province|year|density|timezone|
+-------------------+----+-------+--------+
|     SUMATERA BARAT|2015|    124|     WIB|
|        

Tersedia juga fungsi `like()` yang serupa dengan SQL statement *like*

In [ ]:
df.filter(df.province.like('SU%')).show(5)

+----------------+----+-------+--------+
|        province|year|density|timezone|
+----------------+----+-------+--------+
|  SUMATERA UTARA|2015|    191|     WIB|
|  SUMATERA BARAT|2015|    124|     WIB|
|SUMATERA SELATAN|2015|     88|     WIB|
|  SULAWESI UTARA|2015|    174|    WITA|
| SULAWESI TENGAH|2015|     47|    WITA|
+----------------+----+-------+--------+
only showing top 5 rows



Atau dapat juga menggunakan regex, dengan fungsi `rlike()`

In [ ]:
df.filter(df.province.rlike('[A-Z]*TA$')).show()

+-------------+----+-------+--------+
|     province|year|density|timezone|
+-------------+----+-------+--------+
|  DKI JAKARTA|2015|  15328|     WIB|
|DI YOGYAKARTA|2015|   1174|     WIB|
|  DKI JAKARTA|2014|  15173|     WIB|
|DI YOGYAKARTA|2014|   1161|     WIB|
|  DKI JAKARTA|2013|  15015|     WIB|
|DI YOGYAKARTA|2013|   1147|     WIB|
+-------------+----+-------+--------+



Dapat juga menggunakan filter berdasar list, dengan fungsi `isin()`

In [ ]:
df.filter(df.timezone.isin('WIT','WITA')).show(5)

+-------------------+----+-------+--------+
|           province|year|density|timezone|
+-------------------+----+-------+--------+
|   KALIMANTAN UTARA|2015|      9|    WITA|
| KALIMANTAN SELATAN|2015|    103|    WITA|
|   KALIMANTAN TIMUR|2015|     27|    WITA|
|               BALI|2015|    718|    WITA|
|NUSA TENGGARA BARAT|2015|    260|    WITA|
+-------------------+----+-------+--------+
only showing top 5 rows



Untuk menggunakan beberapa kondisi sekaligus, menggunakan tanda `&` untuk AND dan `|` untuk OR, dengan masing-masing kondisi dilingkupi tanda kurung `()`

In [ ]:
df.filter((df.timezone.isin('WIT','WITA')) & (df.year == 2013)).show(5)

+-------------------+----+-------+--------+
|           province|year|density|timezone|
+-------------------+----+-------+--------+
|   KALIMANTAN UTARA|2013|   null|    WITA|
| KALIMANTAN SELATAN|2013|     99|    WITA|
|   KALIMANTAN TIMUR|2013|     19|    WITA|
|               BALI|2013|    702|    WITA|
|NUSA TENGGARA BARAT|2013|    254|    WITA|
+-------------------+----+-------+--------+
only showing top 5 rows



###2.3 Unique value

Untuk menampilkan nilai unik dari dataframe, digunakan fungsi `distinct()`.

Nilai unik di sini adalah kombinasi nilai dari seluruh kolom.

In [ ]:
df.distinct().show()

Untuk menampilkan nilai unik dari kolom tertentu, tulis nama kolom yang dimaksud sebagai parameter.

In [ ]:
df.select('timezone').distinct().show()
df.select('year','timezone').distinct().show()

####Menghapus duplikasi data

Untuk menghapus record duplikat, gunakan `dropDuplicates(subset)` atau `drop_duplicates(subset)`.

In [ ]:
df.dropDuplicates(['province', 'timezone']).show(100)

###2.4 Agregasi

####Group by column

Untuk mengelompokkan berdasar kolom, gunakan perintah `groupBy('nama_kolom')`

Untuk mengelompokkan berdasar lebih dari 1 kolom, gunakan tanda koma sebagai pemisah nama kolom.

In [ ]:
df.groupBy("timezone")

In [ ]:
df.groupBy("timezone","year")

Perintah `groupBy()` menghasilkan obyek `GroupedData` yang belum bisa ditampilkan.

Biasanya setelah pengelompokan, kita melakukan operasi sumarisasi data. Kita terapkan operasi tersebut pada objek hasil groupBy dengan memanggil fungsi yang dibutuhkan. Misalnya `count()` atau `max('namakolom')`

In [ ]:
df.groupBy('timezone').count().show()
df.groupBy('timezone').max('density').show()

+--------+-----+
|timezone|count|
+--------+-----+
|     WIT|   12|
|     WIB|   54|
|    WITA|   36|
+--------+-----+

+--------+------------+
|timezone|max(density)|
+--------+------------+
|     WIT|          36|
|     WIB|       15328|
|    WITA|         718|
+--------+------------+



Kita juga bisa menggunakan fungsi `agg()` untuk melakukan agregasi. Terutama jika kita ingin melakukan lebih dari 1 operasi agregat.

Kita bisa menggunakan fungsi `alias()` untuk memberi nama kolom hasil agregasi.

In [ ]:
df.groupBy("timezone").agg(F.max('density')).show()
df.groupBy("timezone").agg(F.avg('density').alias('avg_density'), \
                           F.min('density').alias('min_density'), \
                           F.max('density').alias('max_density')).show()

+--------+------------+
|timezone|max(density)|
+--------+------------+
|     WIT|          36|
|     WIB|       15328|
|    WITA|         718|
+--------+------------+

+--------+------------------+-----------+-----------+
|timezone|       avg_density|min_density|max_density|
+--------+------------------+-----------+-----------+
|     WIT|              22.5|          9|         36|
|     WIB|1221.3148148148148|         16|      15328|
|    WITA|157.54285714285714|          8|        718|
+--------+------------------+-----------+-----------+



####Order By

In [ ]:
df.groupBy("timezone") \
  .mean("density") \
  .orderBy("timezone",ascending=False).show()

+--------+------------------+
|timezone|      avg(density)|
+--------+------------------+
|    WITA|157.54285714285714|
|     WIT|              22.5|
|     WIB|1221.3148148148148|
+--------+------------------+



####Agregasi dengan filter / kondisi

In [ ]:
df.groupBy("timezone") \
  .mean("density") \
  .where(df.timezone.contains('WIT')).show()

+--------+------------------+
|timezone|      avg(density)|
+--------+------------------+
|     WIT|              22.5|
|    WITA|157.54285714285714|
+--------+------------------+



####Filter hasil agregat (SQL stat **HAVING**)

Untuk memfilter berdasar hasil agregasi (semacam perintah **HAVING** di SQL), lakukan dalam 2 langkah.
1. Lakukan `groupBy` + `agg` dan beri nama kolom hasil agregat dengan `alias`
2. gunakan fungsi `filter(kondisi)` pada kolom hasil agregasi.

In [ ]:
df_agg = df.groupBy("timezone", "province") \
  .agg(F.avg("density").alias("avg_density")) \
  .where(df.timezone.contains('WIT'))

df_agg.filter(df_agg.avg_density > 50).show()

+--------+-------------------+------------------+
|timezone|           province|       avg_density|
+--------+-------------------+------------------+
|    WITA|     SULAWESI UTARA|             172.0|
|    WITA|          GORONTALO| 99.33333333333333|
|    WITA| KALIMANTAN SELATAN|             101.0|
|    WITA|  SULAWESI TENGGARA| 64.33333333333333|
|    WITA|               BALI|             710.0|
|    WITA|     SULAWESI BARAT|              75.0|
|    WITA|NUSA TENGGARA TIMUR|103.33333333333333|
|    WITA|   SULAWESI SELATAN|180.33333333333334|
|    WITA|NUSA TENGGARA BARAT|             257.0|
+--------+-------------------+------------------+



###2.5 Transformasi DataFrame

####Kolom baru berupa nilai konstan

Untuk menambahkan kolom baru ke dalam dataframe, kita bisa menggunakan perintah `withColumn()`

Sedangkan untuk menambahkan sebuah nilai konstan, kita bisa menggunakan fungsi `lit(nilai_konstan)` dari `pyspark.sql.functions`, yang berfungsi membuat kolom dari nilai literal/konstan.

Misalnya kita ingin menambahkan kolom **status** yang bernilai 1

In [ ]:
df.withColumn('status', F.lit(1)).show(5)

+--------------+----+-------+--------+------+
|      province|year|density|timezone|status|
+--------------+----+-------+--------+------+
|          ACEH|2015|     86|     WIB|     1|
|SUMATERA UTARA|2015|    191|     WIB|     1|
|SUMATERA BARAT|2015|    124|     WIB|     1|
|          RIAU|2015|     73|     WIB|     1|
|         JAMBI|2015|     68|     WIB|     1|
+--------------+----+-------+--------+------+
only showing top 5 rows



####Kolom baru dari kolom yang ada



In [ ]:
df.withColumn('tahun-1', df.year-1).show(5)

+--------------+----+-------+--------+-------+
|      province|year|density|timezone|tahun-1|
+--------------+----+-------+--------+-------+
|          ACEH|2015|     86|     WIB|   2014|
|SUMATERA UTARA|2015|    191|     WIB|   2014|
|SUMATERA BARAT|2015|    124|     WIB|   2014|
|          RIAU|2015|     73|     WIB|   2014|
|         JAMBI|2015|     68|     WIB|   2014|
+--------------+----+-------+--------+-------+
only showing top 5 rows



####Kondisional

Untuk menambahkan kolom berdasar beberapa kondisi, gunakan `when` dan `otherwise` (jika perlu).

Perhatikan bahwa `when` yang pertama adalah fungsi dalam `pyspark.sql.functions`, sedangkan `when` yang berikutnya adalah fungsi `when` pada object kolom (`pyspark.sql.Column`)

Fungsi `otherwise` adalah kondisi *else* atau kondisi selain yang disebutkan pada *when*.

In [ ]:
df.withColumn('timezone1', F.when(df.timezone == 'WIT', 1).
              when(df.timezone == 'WIT', 2).
              otherwise(3)).show(5)

###2.6 Data enrichment - Join


Perintah ntuk melakukan join adalah sebagai berikut :

`df1.join(df2, on=[columname], how=’left’)`

Where :
-    `df1` − Dataframe1.
-    `df2` – Dataframe2.
-    `on` − nama kolom yang akan digunakan untuk join.
-    `how` – type of join needs to be performed – `left`, `right`, `outer`, `inner`. Defaultnya adalah inner join.


Create dataframe yang akan digunakan untuk join

In [ ]:
damdata = (("SUMATERA SELATAN",2),
("SULAWESI TENGAH",2),
("SULAWESI SELATAN",2),
("SUMATERA BARAT",3),
("RIAU",3),
("LAMPUNG",3),
("NUSA TENGGARA TIMUR",4),
("BENGKULU",8),
("SUMATERA UTARA",10),
("JAWA TIMUR",12),
("JAWA TENGAH",35),
("JAWA BARAT",49))

df_dam = spark.createDataFrame(damdata).toDF("province", "dam_num")

df_dam.show()

Join dataframe kepadatan penduduk dengan dataframe jumlah bendungan, berdasarkan nama propinsi.

In [ ]:
df.join(df_dam, on=['province'], how='inner').show()

Untuk melakukan left join, tentukan parameter `how`

In [ ]:
df_2015.join(df_dam, on=['province'], how='left').show(40)

Update history

2302.1956
* _Translate dan lengkapi penjelasan_
* _Menambahkan bab transformasi DataFrame_
* _Code cleanup_
